# Notebook 3:
writing your own library like tensorflow, A MiniFlow library.

Before we start using tensorflow, keras or anyother library, it is best to learn how this library works conceptually. So, to do that we will write our own library - MiniFlow which will work similiarly like tensorFlow. Why this is important? Well, before using this abstractions Isn't it is good to learn how under the hood this library works? forward pass, backprop, derivatives or chain rule?

An intellectually curious mind should know this nutss and bolts and that is why we will write our first Neural Network in from scratch.

## MiniFlow Architecture:

Let's consider how to implement this graph structure in MiniFlow. We'll use a Python class to represent a generic node.

We know that each node might receive input from multiple other nodes. We also know that each node creates a single output, which will likely be passed to other nodes. Let's add two lists: one to store references to the inbound nodes, and the other to store references to the outbound nodes.

In [5]:
class Node(object):
    def __init__(self, inbound_nodes = []):
        #Nodes from which this Node recieves values
        self.inbound_nodes = inbound_nodes
        #Nodes to which this node pass values
        self.outbound_nodes = []
        #for each inbound_nodes add this Node as outbound Node.
        for inbound_node in self.inbound_nodes:
            inbound_node.outbound_nodes.append(self)
        #A calculated final value of this Node
        self.value = None
        
    def forward(self):
        '''
        Forward propagation.
        
        compute the output value based on 'inbound_nodes' and 
        store the result in self.value
        '''
        raise NotImplemented

*While Node defines the basic set of properties that every node holds, only specialized subclasses of will end uo in final graph. So, lets build our first subclass which will calculate the value and hold value.*

In [6]:
class Input(Node):
    def __init__(self):
        #An Input Node has no inbound nodes,
        #so no need to pass anything to the Node instantiator.
        Node.__init__(self)
    
    def forward(self, value=None):
        '''
        since Input node is the node which doesn't have any inbound_nodes
        this forward method will take value as input and set it self.value
        while, other non input Node's forward method will read the value form each inbound_nodes.value
        calculate the resultant and store it in self.value.
        '''
        #overwite the value if one passed in.
        if value is not None:
            self.value = value
        return self.value

*Okay, so far we wrote Input Node which doesn't take any input from other nodes. But itself holds the input to neural network and this value can be set directly by setting Node.vlaue or by passing value to forward method.*

*As we know, in nueral networks there are nodes which takes value from such input nodes or hidden nodes perform actual calculation and save it along with passing it to rest of network.*

*Okay, Lets implement Add Node which will does exactly this.*

In [12]:
class Add(Node):
    def __init__(self, inbound_nodes = []):
        Node.__init__(self, inbound_nodes)
    
    def forward(self):
        """
        Note: this method doesn't has value parameter as we supposed to take
        values from inbound_nodes and perform calculation.
        """
        self.value = 0
        for inbound_node in self.inbound_nodes:
                self.value += inbound_node.value
        
        return self.value

## Forward Propagation:
Like in tensorFlow library, we has to create the computation graph first which gets initialized when we pass values to it and call evaluate. It then checks the computation graph, runs through the computation and give us the output. Similiary, we will implement to menthods in the this library.

`topological_sort() and forward_pass()`, In order to define the network we need to define the order of operations on nodes. Given the input to some node depends on the output of others, we need to flatten this computation graph in such a way that all the nodes gets evaluated first whose inputs are needed to calculate the output of current node.

To resolve this we will implement kahn's Algorithm which will sort the nodes inthe order of their calculation. The input of `topological_sort()` is `feed_dict: a python dict` and the output is `sorted list of nodes`.

Then `forward_pass()` will take this `sorted_nodes` list and do a forward pass on each node and gives back the final `output_node` which will contain the final value of the network.

In [13]:
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

In [14]:
def forward_pass(output_node, sorted_nodes):
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:

        `output_node`: A node in the graph, should be the output node (have no outgoing edges).
        `sorted_nodes`: A topologically sorted list of nodes.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value

In [18]:
"""
This script builds and runs a graph with miniflow.

There is no need to change anything to solve this quiz!

However, feel free to play with the network! Can you also
build a network that solves the equation below?

(x + y) + y
"""

x, y, z = Input(), Input(), Input()

f = Add([x, y, z])

feed_dict = {x: 10, y: 5, z:8}

sorted_nodes = topological_sort(feed_dict)
output = forward_pass(f, sorted_nodes)

# NOTE: because topological_sort sets the values for the `Input` nodes we could also access
# the value for x with x.value (same goes for y).
print("{} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))

10 + 5 = 23 (according to miniflow)


Congratualtions!, on building your first feed forward nueral network.
Next this will be to compare output value:`y'` with true value:`y`, calculate error term and do backprop to adjust the weights to improve the model.


## Learning and loss